In [9]:
import sqlite3
import pandas as pd
import numpy as np

engine = sqlite3.connect('Graph_DB')

graph_df = pd.read_sql('select * from graph_df', engine)

In [10]:
sampled_df = graph_df[['sampled']]
sampler_df = graph_df[['sampler']]
sampler_df = sampler_df.rename(index=str, columns={'sampler':'sampled'})
all_ids = sampled_df.append(sampler_df, ignore_index = True)
all_ids = all_ids.drop_duplicates()
all_ids = all_ids.rename(index=str, columns={'sampled':'song'})
all_ids.to_sql('songs_df', engine, if_exists='replace', index=True)

In [11]:
auth_df = pd.read_sql('select song, 1 as auth from songs_df', engine)
hub_df = pd.read_sql('select song, 1 as hub from songs_df', engine)
auth_df.to_sql('auth_db', engine, if_exists='replace', index=True)
hub_df.to_sql('hub_db', engine, if_exists='replace', index=True)

In [ ]:
beta = 0.15
num_iter = 3
for i in range(0, num_iter):
    hub_l2 = np.sqrt((hub_df['hub']**2).sum())
    auth_l2 = np.sqrt((auth_df['auth']**2).sum())
    hub_df = pd.read_sql('select song, (' + str((1 - beta)) + ' * SUM(auth) + ' + str(beta) + ') as hub from graph_df left join auth_db on ' +
                         'auth_db.song = graph_df.sampler group by song', engine)
    auth_df = pd.read_sql('select song, (' + str((1 - beta)) + ' * SUM(hub) + ' + str(beta) + ') as auth from graph_df left join hub_db on ' +
                         'hub_db.song = graph_df.sampled group by song', engine)
    
    auth_df.to_sql('auth_db', engine, if_exists='replace', index=True)
    hub_df.to_sql('hub_db', engine, if_exists='replace', index=True)
    
    missing_hub_df = pd.read_sql('select songs_df.song, ' + str(beta) + ' as hub from songs_df left join hub_db on ' +
                         'hub_db.song = songs_df.song where hub is null', engine)
    
    missing_auth_df = pd.read_sql('select songs_df.song, ' + str(beta) + ' as auth from songs_df left join auth_db on ' +
                         'auth_db.song = songs_df.song where auth is null', engine)
    hub_df = hub_df.append(missing_hub_df)
    auth_df = auth_df.append(missing_auth_df)
    
    hub_df = hub_df.dropna()
    auth_df = auth_df.dropna()
    hub_l2 = np.sqrt((hub_df['hub']**2).sum())
    auth_l2 = np.sqrt((auth_df['auth']**2).sum())
    hub_df['hub'] = hub_df['hub'] / hub_l2
    auth_df['auth'] = auth_df['auth'] / auth_l2
    auth_df.to_sql('auth_db', engine, if_exists='replace', index=True)
    hub_df.to_sql('hub_db', engine, if_exists='replace', index=True)

In [17]:
auth_df.sort_values(by=['auth'], ascending=0)

,song,auth
124,4015.0,0.495005
86,1807.0,0.047026
577,380408.0,0.035698
122,3897.0,0.035698
416,100617.0,0.035547
957,2428317.0,0.032513
492,167456.0,0.032513
313,54025.0,0.030996
98,2315.0,0.030034
364,71969.0,0.029479


In [18]:
hub_df.sort_values(by=['hub'], ascending=0)

,song,hub
149,4015.0,0.709605
75,761.0,0.038246
156,4731.0,0.037483
161,5047.0,0.036692
127,2315.0,0.035305
262,35694.0,0.031754
263,35705.0,0.031278
155,4579.0,0.031278
223,26909.0,0.029304
160,4886.0,0.029270


In [19]:
auth_df.to_sql('auth_df', engine, if_exists='replace', index=True)
hub_df.to_sql('hub_df', engine, if_exists='replace', index=True)